In [ ]:
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from common.time_layers import *
from dataset import ptb
def to_one_hot(t_train, vocab_size):
    # 使用NumPy的eye函数创建一个独热编码矩阵
    one_hot_matrix = np.eye(vocab_size)[t_train]
    return torch.tensor(one_hot_matrix)

In [ ]:
corpus, word_to_id, id_to_word = ptb.load_data('train')

corpus_size = 1000;

corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

batch_size = 10 
time_size = 5
wordvec_size = 100 #H_in
hidden_size = 100  #H_out

x_train = np.array([corpus[i:i+time_size] for i in range(len(corpus)-time_size) ])
t_train = np.array([corpus[i:i+time_size] for i in range(1, (len(corpus)-time_size+1)) ])

In [ ]:
class torch_rnn(nn.Module):
    def __init__(self, vocab_size, word_vec_size, hidden_size):
        super().__init__()
        self.V = vocab_size;
        self.H_in = word_vec_size
        self.H_out = hidden_size
        
        self.embed = nn.Embedding(vocab_size,word_vec_size)
        
        self.rnn = nn.RNN(word_vec_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        nn.init.xavier_normal_(self.rnn.weight_hh_l0)
        nn.init.xavier_normal_(self.rnn.weight_ih_l0)
        
        
    def forward(self,seq, h_prev):
        N , T  = seq.size()
        
        word_embed = self.embed(seq);
        out, h_prev = self.rnn(word_embed, h_prev)
        out = out.reshape(N*T,-1);
        
        out= self.linear(out).reshape(N,T,vocab_size)
        
        return out , h_prev
    

In [ ]:
model = torch_rnn( vocab_size, wordvec_size , hidden_size )
h_prev = torch.zeros((1,batch_size,hidden_size))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),0.001)

In [ ]:
l=[]
for i in range(2500):
    mini_batch_mask =  np.random.choice(len(x_train),batch_size)
    x_train_mini_batch = torch.tensor(x_train[mini_batch_mask])
    t_train_mini_batch = torch.tensor(t_train[mini_batch_mask])
    t_train_mini_batch = to_one_hot(t_train_mini_batch,vocab_size)
    
    out , h_prev = model(x_train_mini_batch,h_prev)
    h_prev = h_prev.detach()
    out = out.reshape(50,-1)
    t_train_mini_batch = t_train_mini_batch.reshape(50,-1)
    loss = criterion(out,t_train_mini_batch) #要保证第二个维度是 vocab_size
    model.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i % 100 ==0:
        print('{},{}'.format(i,loss.item()))
        l.append(loss.item())
        